<a href="https://colab.research.google.com/github/aditeyabaral/gpt2-implementation/blob/main/huggingface_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers Implementation

# Installing and Setting up Environment

In [1]:
!pip install transformers

     |████████████████████████████████| 1.1MB 3.5MB/s 
     |████████████████████████████████| 890kB 17.9MB/s 
     |████████████████████████████████| 3.0MB 26.4MB/s 
     |████████████████████████████████| 1.1MB 47.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=0ee5c5981bd30edb1bdd4136de8e67c774503e586b14ca67c6a85906b24eb59f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, pipeline
from transformers import GPT2Config, GPT2LMHeadModel

# Loading Training Data

In [3]:
!curl -L -O https://raw.githubusercontent.com/aditeyabaral/gpt2-implementation/main/Simpsons.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8998k  100 8998k    0     0  14.4M      0 --:--:-- --:--:-- --:--:-- 14.4M


In [4]:
df = pd.read_csv("Simpsons.csv")
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)
df.reset_index(inplace = True)
df.drop(columns = "index", inplace = True)
print(df.shape)
df.head()

(126646, 2)


,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [5]:
def character_slice(character):
  if character in np.unique(df[["raw_character_text"]]):
    return df[df["raw_character_text"] == character]

In [6]:
CHARACTER = "Miss Hoover"

character_df = character_slice(CHARACTER)
print(character_df.shape)
character_df.head()

(154, 2)


,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
4426,Miss Hoover,"Thank you, Ralph. Very graphic. Lisa Simpson? ..."
4428,Miss Hoover,Dear God!
4785,Miss Hoover,I question the educational value of this assem...


## Adding delimiters

This is optional - allows for retention of training data structure

In [7]:
train_text = "\n".join(character_df["spoken_words"].values)

In [8]:
with open("corpus.txt","w") as f:
  f.write(train_text)

# Training

In [9]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
train_path = "/content/corpus.txt"
train_data = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=4)
collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [10]:
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)

In [ ]:
model = GPT2LMHeadModel(config)
training_args=TrainingArguments(output_dir="/content/output",overwrite_output_dir=True,num_train_epochs=10,
                                per_device_train_batch_size=1,per_device_eval_batch_size=1,eval_steps=100,save_steps=5000,warmup_steps=100,gradient_accumulation_steps=1)
torch.cuda.empty_cache()
trainer = Trainer(model=model, args=training_args, data_collator=collator, train_dataset=train_data, prediction_loss_only=True)
trainer.train()
trainer.save_model()

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:267: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead.
  FutureWarning,


# Generation

In [ ]:
generator = pipeline("text-generation",model="/content/output",tokenizer="gpt2",config={'max-length':280})
generated_text = generator("Bart")[0]['generated_text']
print(generated_text)